# Data Cleaning

This notebook will clean up the data for analysis.

## Setup

Get the data from the following sources (use "Export" in the top right, export as csv):
* [Police Call Data](https://data.seattle.gov/Public-Safety/Call-Data/33kz-ixgy) 
* [Fremont Bridge Bicycle Counter](https://data.seattle.gov/Transportation/Fremont-Bridge-Bicycle-Counter/65db-xm6k)
* [NW 58th St Greenway at 22nd Ave NW Bicycle Counter](https://data.seattle.gov/Transportation/NW-58th-St-Greenway-at-22nd-Ave-NW-Bicycle-Counter/47yq-6ugv)

Put the csv files in a directory named `data/` next to this notebook and be sure the file names listed below are correct.

```
.
├── data
│   ├── Call_Data.csv
│   ├── Fremont_Bridge_Bicycle_Counter.csv
│   └── NW_58th_St_Greenway_at_22nd_Ave_NW_Bicycle_Counter.csv
├── Data Cleaning.ipynb
```

Run this notebook and it should output the cleaned data.

In [1]:
# Imports
from os import path
import os
import pandas as pd

## Load the Frames

This will load the frames from the CSV file. It may take a minute, especially for the police call data, so give it time.

In [2]:
# Files
call_file = path.join("data", "Call_Data.csv")
fremont_file = path.join("data", "Fremont_Bridge_Bicycle_Counter.csv")
greenway_file = path.join("data", "NW_58th_St_Greenway_at_22nd_Ave_NW_Bicycle_Counter.csv")

In [3]:
# This frame is very large, so it's in it's own cell so you can run it once and not multiple times
call_frame = pd.read_csv(call_file)

In [4]:
# Load the Bike Counter frames
fremont_frame = pd.read_csv(fremont_file)
greenway_frame = pd.read_csv(greenway_file)

## Rename Columns

We'll rename them so they're easier to reference.

In [5]:
# Rename police call frame columns
call_column_names = {
    'CAD Event Number': 'cad_num',
    'Event Clearance Description': 'clearance_desc',
    'Call Type': 'call_type',
    'Priority': 'priority',
    'Initial Call Type': 'initial_call_type',
    'Final Call Type': 'final_call_type',
    'Original Time Queued': 'time_queued',
    'Arrived Time': 'arrived_time',
    'Precinct': 'precinct',
    'Sector': 'sector',
    'Beat': 'beat',
    'Blurred_Longitude': 'blurred_long',
    'Blurred_Latitude': 'blurred_lat'
}
_ = call_frame.rename(columns = call_column_names, inplace = True)

In [6]:
fremont_column_names = {
    'Date': 'date',
    'Fremont Bridge Sidewalks, south of N 34th St': 'total',
    'Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk': 'east',
    'Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk': 'west'
}

fremont_frame.rename(columns = fremont_column_names, inplace = True)

In [7]:
greenway_column_names = {
    'Date': 'date',
    'NW 58th St Greenway st 22nd Ave NW Total': 'total',
    'East': 'east',
    'West': 'west'
}
_ = greenway_frame.rename(columns = greenway_column_names, inplace = True)

## Filter by Dates

First we should get the date ranges for all the frames. We only need to keep the police call data that overlaps with the bike counter date ranges.

### Parse Dates and Times

In [8]:
# Parse the date columns into date types
# Also split the date and time

# We only want to parse once, this will prevent it from parsing again if we already have
if 'time' not in fremont_frame.columns:
    fremont_frame['date'] = pd.to_datetime(fremont_frame['date'], format='%m/%d/%Y %I:%M:%S %p')
    fremont_frame['time'] = fremont_frame['date'].dt.time
    fremont_frame['date'] = fremont_frame['date'].dt.date

if 'time' not in greenway_frame.columns:
    greenway_frame['date'] = pd.to_datetime(greenway_frame['date'], format='%m/%d/%Y %I:%M:%S %p')
    greenway_frame['time'] = greenway_frame['date'].dt.time
    greenway_frame['date'] = greenway_frame['date'].dt.date

In [9]:
# Parse the call frame arrived_time to a date time type
# Keep this in a separate cell as above because it takes a second

# Same as above, split the date and time for arrived_date
if 'arrived_date' not in call_frame.columns:
    call_frame['arrived_time'] = pd.to_datetime(call_frame['arrived_time'], format='%m/%d/%Y %I:%M:%S %p')
    call_frame['arrived_date'] = call_frame['arrived_time'].dt.date
    call_frame['arrived_time'] = call_frame['arrived_time'].dt.time

In [10]:
# Get date ranges for both
fremont_range  = ( fremont_frame['date'].min(),  fremont_frame['date'].max())
greenway_range = (greenway_frame['date'].min(), greenway_frame['date'].max())

In [11]:
# Just asserts that the arrived_date of the police call is within the provided date_range
between_dates = lambda frame, date_range: (frame['arrived_date'] >= date_range[0]) & (frame['arrived_date'] <= date_range[1])

fremont_calls = call_frame[between_dates(call_frame, fremont_range)]
greenway_calls = call_frame[between_dates(call_frame, greenway_range)]

# Assert that we didn't mess up the filter logic
assert len(fremont_calls) > 0
assert len(greenway_calls) > 0

## Filter by Location

We only need to keep the police calls within a certain radius of the bike counters.

Areas that police patrol are split into precint -> sector -> beat. The beat is a letter, then number, where the letter corresponds to the sector.

You can find a ["beat map" of Seattle here](https://www.seattle.gov/police/about-us/police-locations/precinct-locator)

Looking at the Fremont Bridge, it's in beat Q2, but we'll include neighboring beats that are easy to access from the bridge as well. We'll do the same for the Greenway counter.

In [12]:
fremont_beats = ['Q2', 'B2', 'B3', 'D2']
greenway_beats = ['B1', 'B2', 'B3', 'J2', 'J3']

fremont_calls = fremont_calls.loc[fremont_calls['beat'].isin(fremont_beats)]
greenway_calls = greenway_calls.loc[greenway_calls['beat'].isin(greenway_beats)]

## Write cleaned data to file

The new filtered and cleaned frames will be written to the `data/cleaned/` directory, so we can load them from there and leave the original data alone. You can always delete the cleaned data and re-run this script to regenerate it.

In [13]:
# Shorthand to get a path to the output directory
out_file = lambda filename: path.join('data', 'cleaned', filename)

# Create the 'cleaned/' directory if it doesn't exist
if not path.exists(out_file('')):
    os.makedirs(out_file(''))

In [14]:
# Write the fremont calls
fremont_calls.to_csv(out_file('fremont_calls.csv'), index = False)

In [15]:
# Write the greenway calls
greenway_calls.to_csv(out_file('greenway_calls.csv'), index = False)

In [16]:
# Write the fremont bike counter data
fremont_frame.to_csv(out_file('fremont_bikes.csv'), index = False)

In [17]:
# Write the greenway bike counter data
greenway_frame.to_csv(out_file('greenway_bikes.csv'), index = False)

# Summary

Now, the files we have are:

```
.
├── data
│   ├── cleaned
│   │   ├── fremont_bikes.csv
│   │   ├── fremont_calls.csv
│   │   ├── greenway_bikes.csv
│   │   └── greenway_calls.csv
│   ├── Call_Data.csv
│   ├── Fremont_Bridge_Bicycle_Counter.csv
│   └── NW_58th_St_Greenway_at_22nd_Ave_NW_Bicycle_Counter.csv
```

We can now work with the cleaned data files:

* `data/cleaned/fremont_bikes.csv` is basically the original Fremont bike counter data set, but with renamed/split columns
* `data/cleaned/greenway_bikes.csv` is the same as above, but for the Greenway counter
* `data/cleaned/fremont_calls.csv` is all the police call data that overlaps with the Fremont bike counter dates, and is in the police beat or neighboring beats. Again, with renamed/split columns.
* `data/cleaned/greenway_calls.csv` is the same as above, but for the Greenway counter.

The original `csv` files in `data/` are untouched. We can always regenerate the cleaned files using this notebook, so the originals shouldn't be edited.

Going forward, we should use the cleaned data like this:

In [22]:
# This is how we'd start a new notebook
# Also note that we have to re-parse date/time info like we did above, since it's stored as plain text in the csv file
fremont_calls = pd.read_csv('data/cleaned/fremont_calls.csv')
fremont_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314737 entries, 0 to 314736
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cad_num            314737 non-null  int64  
 1   clearance_desc     314737 non-null  object 
 2   call_type          314737 non-null  object 
 3   priority           314705 non-null  float64
 4   initial_call_type  314737 non-null  object 
 5   final_call_type    314737 non-null  object 
 6   time_queued        314737 non-null  object 
 7   arrived_time       314737 non-null  object 
 8   precinct           314737 non-null  object 
 9   sector             314620 non-null  object 
 10  beat               314737 non-null  object 
 11  blurred_long       314737 non-null  float64
 12  blurred_lat        314737 non-null  float64
 13  arrived_date       314737 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 33.6+ MB


# What's Next

I think a good next step is to summarize the police call data by hour. We have hourly reports for the bike counters, so we could reduce the police call data into hourly reports as well so they can be compared.

This might be something like summarizing the amount of police calls in the hour, the most common type of police response in that hour, the percentage of violent crimes in that hour, etc. Then we can look for correlations. For example, we create a new data frame that has the amount of traffic related police responses per hour, and see if there's correlation between traffic responses and bike traffic.

## Ideas for hourly reports

* Most common type of police response in that hour (this would be a classification problem)
* Call time to response time delay (would have to parse the original time and arrival time, I just did arrival time in this notebook
* Number of violent crime responses in that hour
* Number of traffic related calls in that hour
* Number of calls overall in that area